In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import gzip
import pickle
import librosa

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
pickle_off=gzip.open("drive/My Drive/Comp562/Synth1_onenote.data","rb")
emp=pickle.load(pickle_off)

In [39]:
sampling_rate=44100
nsamples=1000;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
wlength=24;
targets = 255;
ntrain=600;
timeslices=131;
nfeatures=timeslices*(nmfccs+nchroma+6+1)

X=torch.empty(nsamples,timeslices*(nmfccs+nchroma+6+1))
y=torch.empty(nsamples,targets)
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.array(emp[i][0]), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.array(emp[i][0]), sr=sampling_rate,n_chroma=nchroma)
    toneTemp=librosa.feature.tonnetz(y=np.array(emp[i][0]), sr=sampling_rate)
    spectralBandTemp=librosa.feature.spectral_bandwidth(y=np.array(emp[i][0]))
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.array(emp[i][0]),n_bands=nbands)
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    X[i,:]=torch.tensor(feature.flatten())
    y[i,:]=torch.tensor(([emp[i][1][j][1] for j in range(0,targets)]))

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [0]:
with open('drive/My Drive/COMP562/Synth1ProcessedPatchesMLP.data', 'wb') as f:
  pickle.dump([X, y], f)

In [0]:
def exportResultsPytorch(testIndex,yDataOriginal,Results,fileName):
  predictions=yDataOriginal[list(testIndex),:]
  with open(fileName, 'wb') as f:
    pickle.dump([y[list(testIndex),:].numpy(), Results.detach().numpy()], f)
  f.close()

In [0]:
np.random.seed(seed=5)
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex
X_train=X[list(trainIndex),:]
y_train=y[list(trainIndex),:]
X_test=X[list(testIndex),:]
y_test=y[list(testIndex),:]

In [0]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=2000, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(10000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/Synth1MLP0")

diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1]))
results0 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/Synth1Results0.data')

In [60]:
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))

tensor(0.5375, grad_fn=<SqrtBackward>)


In [62]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden3 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden4 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden5 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden6 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden7 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden8 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden9 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden10 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden11 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden12 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = torch.tanh(self.hidden3(x))
		x = F.relu(self.hidden4(x))
		x = F.relu(self.hidden5(x))
		x = F.relu(self.hidden6(x))
		x = F.relu(self.hidden7(x))
		x = F.relu(self.hidden8(x))
		x = F.relu(self.hidden9(x))
		x = torch.tanh(self.hidden10(x))
		x = F.relu(self.hidden11(x))
		x = F.relu(self.hidden12(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=20, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(50000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/Synth1MLP1")


diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results1 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/Synth1Results1.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=20, bias=True)
  (hidden2): Linear(in_features=20, out_features=20, bias=True)
  (hidden3): Linear(in_features=20, out_features=20, bias=True)
  (hidden4): Linear(in_features=20, out_features=20, bias=True)
  (hidden5): Linear(in_features=20, out_features=20, bias=True)
  (hidden6): Linear(in_features=20, out_features=20, bias=True)
  (hidden7): Linear(in_features=20, out_features=20, bias=True)
  (hidden8): Linear(in_features=20, out_features=20, bias=True)
  (hidden9): Linear(in_features=20, out_features=20, bias=True)
  (hidden10): Linear(in_features=20, out_features=20, bias=True)
  (hidden11): Linear(in_features=20, out_features=20, bias=True)
  (hidden12): Linear(in_features=20, out_features=20, bias=True)
  (predict): Linear(in_features=20, out_features=255, bias=True)
)
Loss=0.6034
Loss=0.6010
Loss=0.5987
Loss=0.5965
Loss=0.5944
Loss=0.5923
Loss=0.5903
Loss=0.5883
Loss=0.5864
Loss=0.5844
Loss=0.5826
Loss=0.5807
Loss=0.5789

In [0]:
pickle_off=gzip.open("drive/My Drive/Comp562/ComboF_onenote.data","rb")
emp=pickle.load(pickle_off)

In [0]:
sampling_rate=44100
nsamples=1000;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
wlength=24;
targets = 42;
ntrain=600;
timeslices=131;
nfeatures=timeslices*(nmfccs+nchroma+6+1)

X=torch.empty(nsamples,timeslices*(nmfccs+nchroma+6+1))
y=torch.empty(nsamples,targets)
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.array(emp[i][0]), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.array(emp[i][0]), sr=sampling_rate,n_chroma=nchroma)
    toneTemp=librosa.feature.tonnetz(y=np.array(emp[i][0]), sr=sampling_rate)
    spectralBandTemp=librosa.feature.spectral_bandwidth(y=np.array(emp[i][0]))
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.array(emp[i][0]),n_bands=nbands)
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    X[i,:]=torch.tensor(feature.flatten())
    y[i,:]=torch.tensor(([emp[i][1][j][1] for j in range(0,targets)]))

In [0]:
np.random.seed(seed=5)
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex
X_train=X[list(trainIndex),:]
y_train=y[list(trainIndex),:]
X_test=X[list(testIndex),:]
y_test=y[list(testIndex),:]

In [69]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=2000, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(10000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/ComboFMLP0")

diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results0 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/ComboFResults0.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=2000, bias=True)
  (hidden2): Linear(in_features=2000, out_features=2000, bias=True)
  (predict): Linear(in_features=2000, out_features=42, bias=True)
)
Loss=0.5037
Loss=0.4913
Loss=0.4791
Loss=0.4671
Loss=0.4553
Loss=0.4435
Loss=0.4317
Loss=0.4199
Loss=0.4081
Loss=0.3962
Loss=0.3842
Loss=0.3721
Loss=0.3600
Loss=0.3478
Loss=0.3358
Loss=0.3239
Loss=0.3123
Loss=0.3013
Loss=0.2908
Loss=0.2813


In [71]:
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))

tensor(0.2725, grad_fn=<SqrtBackward>)


In [72]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden3 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden4 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden5 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden6 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden7 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden8 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden9 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden10 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden11 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden12 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = torch.tanh(self.hidden3(x))
		x = F.relu(self.hidden4(x))
		x = F.relu(self.hidden5(x))
		x = F.relu(self.hidden6(x))
		x = F.relu(self.hidden7(x))
		x = F.relu(self.hidden8(x))
		x = F.relu(self.hidden9(x))
		x = torch.tanh(self.hidden10(x))
		x = F.relu(self.hidden11(x))
		x = F.relu(self.hidden12(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=20, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(50000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/ComboFMLP1")


diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results1 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/ComboFResults1.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=20, bias=True)
  (hidden2): Linear(in_features=20, out_features=20, bias=True)
  (hidden3): Linear(in_features=20, out_features=20, bias=True)
  (hidden4): Linear(in_features=20, out_features=20, bias=True)
  (hidden5): Linear(in_features=20, out_features=20, bias=True)
  (hidden6): Linear(in_features=20, out_features=20, bias=True)
  (hidden7): Linear(in_features=20, out_features=20, bias=True)
  (hidden8): Linear(in_features=20, out_features=20, bias=True)
  (hidden9): Linear(in_features=20, out_features=20, bias=True)
  (hidden10): Linear(in_features=20, out_features=20, bias=True)
  (hidden11): Linear(in_features=20, out_features=20, bias=True)
  (hidden12): Linear(in_features=20, out_features=20, bias=True)
  (predict): Linear(in_features=20, out_features=42, bias=True)
)
Loss=0.5230
Loss=0.5071
Loss=0.4922
Loss=0.4780
Loss=0.4647
Loss=0.4518
Loss=0.4393
Loss=0.4271
Loss=0.4149
Loss=0.4026
Loss=0.3902
Loss=0.3776
Loss=0.3649


In [0]:
pickle_off=gzip.open("drive/My Drive/Comp562/TAL-NoiseMaker_onenote.data","rb")
emp=pickle.load(pickle_off)

In [74]:
sampling_rate=44100
nsamples=1000;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
wlength=24;
targets = 91;
ntrain=600;
timeslices=131;
nfeatures=timeslices*(nmfccs+nchroma+6+1)

X=torch.empty(nsamples,timeslices*(nmfccs+nchroma+6+1))
y=torch.empty(nsamples,targets)
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.array(emp[i][0]), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.array(emp[i][0]), sr=sampling_rate,n_chroma=nchroma)
    toneTemp=librosa.feature.tonnetz(y=np.array(emp[i][0]), sr=sampling_rate)
    spectralBandTemp=librosa.feature.spectral_bandwidth(y=np.array(emp[i][0]))
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.array(emp[i][0]),n_bands=nbands)
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    X[i,:]=torch.tensor(feature.flatten())
    y[i,:]=torch.tensor(([emp[i][1][j][1] for j in range(0,targets)]))

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [0]:
np.random.seed(seed=5)
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex
X_train=X[list(trainIndex),:]
y_train=y[list(trainIndex),:]
X_test=X[list(testIndex),:]
y_test=y[list(testIndex),:]

In [76]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=2000, n_output=42)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(10000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/TALMLP0")

diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results0 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/TALResults0.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=2000, bias=True)
  (hidden2): Linear(in_features=2000, out_features=2000, bias=True)
  (predict): Linear(in_features=2000, out_features=42, bias=True)
)
Loss=0.5040
Loss=0.4915
Loss=0.4793
Loss=0.4671
Loss=0.4551
Loss=0.4432
Loss=0.4312
Loss=0.4191
Loss=0.4070
Loss=0.3948
Loss=0.3825
Loss=0.3701
Loss=0.3576
Loss=0.3451
Loss=0.3327
Loss=0.3206
Loss=0.3088
Loss=0.2976
Loss=0.2872
Loss=0.2777
tensor(0.2692, grad_fn=<SqrtBackward>)


In [77]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden3 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden4 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden5 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden6 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden7 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden8 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden9 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden10 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden11 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden12 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = torch.tanh(self.hidden3(x))
		x = F.relu(self.hidden4(x))
		x = F.relu(self.hidden5(x))
		x = F.relu(self.hidden6(x))
		x = F.relu(self.hidden7(x))
		x = F.relu(self.hidden8(x))
		x = F.relu(self.hidden9(x))
		x = torch.tanh(self.hidden10(x))
		x = F.relu(self.hidden11(x))
		x = F.relu(self.hidden12(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=20, n_output=42)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(50000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/TALMLP1")


diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results1 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/TALResults1.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=20, bias=True)
  (hidden2): Linear(in_features=20, out_features=20, bias=True)
  (hidden3): Linear(in_features=20, out_features=20, bias=True)
  (hidden4): Linear(in_features=20, out_features=20, bias=True)
  (hidden5): Linear(in_features=20, out_features=20, bias=True)
  (hidden6): Linear(in_features=20, out_features=20, bias=True)
  (hidden7): Linear(in_features=20, out_features=20, bias=True)
  (hidden8): Linear(in_features=20, out_features=20, bias=True)
  (hidden9): Linear(in_features=20, out_features=20, bias=True)
  (hidden10): Linear(in_features=20, out_features=20, bias=True)
  (hidden11): Linear(in_features=20, out_features=20, bias=True)
  (hidden12): Linear(in_features=20, out_features=20, bias=True)
  (predict): Linear(in_features=20, out_features=42, bias=True)
)
Loss=0.5042
Loss=0.4918
Loss=0.4793
Loss=0.4669
Loss=0.4556
Loss=0.4442
Loss=0.4326
Loss=0.4207
Loss=0.4085
Loss=0.3959
Loss=0.3830
Loss=0.3699
Loss=0.3564


In [0]:
pickle_off=gzip.open("drive/My Drive/Comp562/Obxd_onenote.data","rb")
emp=pickle.load(pickle_off)

In [79]:
sampling_rate=44100
nsamples=1000;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
wlength=24;
targets = 80;
ntrain=600;
timeslices=131;
nfeatures=timeslices*(nmfccs+nchroma+6+1)

X=torch.empty(nsamples,timeslices*(nmfccs+nchroma+6+1))
y=torch.empty(nsamples,targets)
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.array(emp[i][0]), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.array(emp[i][0]), sr=sampling_rate,n_chroma=nchroma)
    toneTemp=librosa.feature.tonnetz(y=np.array(emp[i][0]), sr=sampling_rate)
    spectralBandTemp=librosa.feature.spectral_bandwidth(y=np.array(emp[i][0]))
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.array(emp[i][0]),n_bands=nbands)
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    X[i,:]=torch.tensor(feature.flatten())
    y[i,:]=torch.tensor(([emp[i][1][j][1] for j in range(0,targets)]))

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [0]:
np.random.seed(seed=5)
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex
X_train=X[list(trainIndex),:]
y_train=y[list(trainIndex),:]
X_test=X[list(testIndex),:]
y_test=y[list(testIndex),:]

In [81]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=2000, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(10000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/ObxdMLP0")

diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results0 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/ObxdResults0.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=2000, bias=True)
  (hidden2): Linear(in_features=2000, out_features=2000, bias=True)
  (predict): Linear(in_features=2000, out_features=80, bias=True)
)
Loss=0.5721
Loss=0.5658
Loss=0.5595
Loss=0.5533
Loss=0.5471
Loss=0.5409
Loss=0.5347
Loss=0.5284
Loss=0.5221
Loss=0.5158
Loss=0.5094
Loss=0.5029
Loss=0.4962
Loss=0.4895
Loss=0.4826
Loss=0.4756
Loss=0.4684
Loss=0.4610
Loss=0.4534
Loss=0.4456
tensor(0.4366, grad_fn=<SqrtBackward>)


In [82]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden3 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden4 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden5 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden6 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden7 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden8 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden9 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden10 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden11 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden12 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = torch.tanh(self.hidden3(x))
		x = F.relu(self.hidden4(x))
		x = F.relu(self.hidden5(x))
		x = F.relu(self.hidden6(x))
		x = F.relu(self.hidden7(x))
		x = F.relu(self.hidden8(x))
		x = F.relu(self.hidden9(x))
		x = torch.tanh(self.hidden10(x))
		x = F.relu(self.hidden11(x))
		x = F.relu(self.hidden12(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=20, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(50000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/ObxdMLP1")


diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results1 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/ObxdResults1.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=20, bias=True)
  (hidden2): Linear(in_features=20, out_features=20, bias=True)
  (hidden3): Linear(in_features=20, out_features=20, bias=True)
  (hidden4): Linear(in_features=20, out_features=20, bias=True)
  (hidden5): Linear(in_features=20, out_features=20, bias=True)
  (hidden6): Linear(in_features=20, out_features=20, bias=True)
  (hidden7): Linear(in_features=20, out_features=20, bias=True)
  (hidden8): Linear(in_features=20, out_features=20, bias=True)
  (hidden9): Linear(in_features=20, out_features=20, bias=True)
  (hidden10): Linear(in_features=20, out_features=20, bias=True)
  (hidden11): Linear(in_features=20, out_features=20, bias=True)
  (hidden12): Linear(in_features=20, out_features=20, bias=True)
  (predict): Linear(in_features=20, out_features=80, bias=True)
)
Loss=0.5940
Loss=0.5873
Loss=0.5807
Loss=0.5745
Loss=0.5684
Loss=0.5623
Loss=0.5563
Loss=0.5504
Loss=0.5444
Loss=0.5384
Loss=0.5324
Loss=0.5264
Loss=0.5202


In [0]:
pickle_off=gzip.open("drive/My Drive/Comp562/Tunefish4_onenote.data","rb")
emp=pickle.load(pickle_off)

In [84]:
sampling_rate=44100
nsamples=1000;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
wlength=24;
targets = 112;
ntrain=600;
timeslices=131;
nfeatures=timeslices*(nmfccs+nchroma+6+1)

X=torch.empty(nsamples,timeslices*(nmfccs+nchroma+6+1))
y=torch.empty(nsamples,targets)
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.array(emp[i][0]), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.array(emp[i][0]), sr=sampling_rate,n_chroma=nchroma)
    toneTemp=librosa.feature.tonnetz(y=np.array(emp[i][0]), sr=sampling_rate)
    spectralBandTemp=librosa.feature.spectral_bandwidth(y=np.array(emp[i][0]))
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.array(emp[i][0]),n_bands=nbands)
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    X[i,:]=torch.tensor(feature.flatten())
    y[i,:]=torch.tensor(([emp[i][1][j][1] for j in range(0,targets)]))

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [0]:
np.random.seed(seed=5)
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex
X_train=X[list(trainIndex),:]
y_train=y[list(trainIndex),:]
X_test=X[list(testIndex),:]
y_test=y[list(testIndex),:]

In [86]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=2000, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(10000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/TuneMLP0")

diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results0 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/TuneResults0.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=2000, bias=True)
  (hidden2): Linear(in_features=2000, out_features=2000, bias=True)
  (predict): Linear(in_features=2000, out_features=112, bias=True)
)
Loss=0.5766
Loss=0.5720
Loss=0.5674
Loss=0.5629
Loss=0.5584
Loss=0.5539
Loss=0.5494
Loss=0.5448
Loss=0.5403
Loss=0.5357
Loss=0.5310
Loss=0.5263
Loss=0.5216
Loss=0.5168
Loss=0.5118
Loss=0.5068
Loss=0.5017
Loss=0.4965
Loss=0.4912
Loss=0.4857
tensor(0.4815, grad_fn=<SqrtBackward>)


In [87]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden3 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden4 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden5 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden6 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden7 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden8 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden9 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden10 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden11 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden12 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = torch.tanh(self.hidden3(x))
		x = F.relu(self.hidden4(x))
		x = F.relu(self.hidden5(x))
		x = F.relu(self.hidden6(x))
		x = F.relu(self.hidden7(x))
		x = F.relu(self.hidden8(x))
		x = F.relu(self.hidden9(x))
		x = torch.tanh(self.hidden10(x))
		x = F.relu(self.hidden11(x))
		x = F.relu(self.hidden12(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=20, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(50000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/TuneMLP1")


diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results1 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/TuneResults1.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=20, bias=True)
  (hidden2): Linear(in_features=20, out_features=20, bias=True)
  (hidden3): Linear(in_features=20, out_features=20, bias=True)
  (hidden4): Linear(in_features=20, out_features=20, bias=True)
  (hidden5): Linear(in_features=20, out_features=20, bias=True)
  (hidden6): Linear(in_features=20, out_features=20, bias=True)
  (hidden7): Linear(in_features=20, out_features=20, bias=True)
  (hidden8): Linear(in_features=20, out_features=20, bias=True)
  (hidden9): Linear(in_features=20, out_features=20, bias=True)
  (hidden10): Linear(in_features=20, out_features=20, bias=True)
  (hidden11): Linear(in_features=20, out_features=20, bias=True)
  (hidden12): Linear(in_features=20, out_features=20, bias=True)
  (predict): Linear(in_features=20, out_features=112, bias=True)
)
Loss=0.5820
Loss=0.5771
Loss=0.5725
Loss=0.5682
Loss=0.5640
Loss=0.5599
Loss=0.5560
Loss=0.5523
Loss=0.5486
Loss=0.5450
Loss=0.5414
Loss=0.5379
Loss=0.5344

In [0]:
pickle_off=gzip.open("drive/My Drive/Comp562/Dexed_onenote.data","rb")
emp=pickle.load(pickle_off)

In [89]:
sampling_rate=44100
nsamples=1000;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
wlength=24;
targets = 155;
ntrain=600;
timeslices=131;
nfeatures=timeslices*(nmfccs+nchroma+6+1)

X=torch.empty(nsamples,timeslices*(nmfccs+nchroma+6+1))
y=torch.empty(nsamples,targets)
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.array(emp[i][0]), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.array(emp[i][0]), sr=sampling_rate,n_chroma=nchroma)
    toneTemp=librosa.feature.tonnetz(y=np.array(emp[i][0]), sr=sampling_rate)
    spectralBandTemp=librosa.feature.spectral_bandwidth(y=np.array(emp[i][0]))
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.array(emp[i][0]),n_bands=nbands)
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    X[i,:]=torch.tensor(feature.flatten())
    y[i,:]=torch.tensor(([emp[i][1][j][1] for j in range(0,targets)]))

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [0]:
np.random.seed(seed=5)
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex
X_train=X[list(trainIndex),:]
y_train=y[list(trainIndex),:]
X_test=X[list(testIndex),:]
y_test=y[list(testIndex),:]

In [91]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=2000, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(10000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/DexedMLP0")

diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results0 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/DexedResults0.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=2000, bias=True)
  (hidden2): Linear(in_features=2000, out_features=2000, bias=True)
  (predict): Linear(in_features=2000, out_features=155, bias=True)
)
Loss=0.5854
Loss=0.5820
Loss=0.5786
Loss=0.5752
Loss=0.5718
Loss=0.5684
Loss=0.5650
Loss=0.5616
Loss=0.5582
Loss=0.5547
Loss=0.5512
Loss=0.5477
Loss=0.5442
Loss=0.5406
Loss=0.5369
Loss=0.5332
Loss=0.5294
Loss=0.5256
Loss=0.5216
Loss=0.5176
tensor(0.5167, grad_fn=<SqrtBackward>)


In [92]:
class Net(torch.nn.Module):
	def __init__(self, n_feature, n_hidden, n_output):
		super(Net, self).__init__()
		self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
		self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden3 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden4 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden5 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden6 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden7 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden8 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden9 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden10 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden11 = torch.nn.Linear(n_hidden, n_hidden)
		self.hidden12 = torch.nn.Linear(n_hidden, n_hidden)
		self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
	def forward(self, x):
		x = F.relu(self.hidden1(x))      # activation function for hidden layer
		x = F.relu(self.hidden2(x))
		x = torch.tanh(self.hidden3(x))
		x = F.relu(self.hidden4(x))
		x = F.relu(self.hidden5(x))
		x = F.relu(self.hidden6(x))
		x = F.relu(self.hidden7(x))
		x = F.relu(self.hidden8(x))
		x = F.relu(self.hidden9(x))
		x = torch.tanh(self.hidden10(x))
		x = F.relu(self.hidden11(x))
		x = F.relu(self.hidden12(x))
		x = self.predict(x)             # linear output
		return x
		
net = Net(n_feature=nfeatures, n_hidden=20, n_output=targets)     # define the network
net.cuda()
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(50000):
	prediction = net(x_train.cuda())     # input x and predict based on x
	loss = torch.sqrt(loss_func(prediction.cuda(), y_train.cuda()))     # must be (1. nn output, 2. target)
	optimizer.zero_grad()   # clear gradients for next train
	loss.backward()         # backpropagation, compute gradients
	optimizer.step()        # apply gradients
	if t % 500 == 0:
		# plot and show learning process
		print('Loss=%.4f' % loss.cpu().data.numpy())
    
torch.save(net.state_dict(),"drive/My Drive/Comp562/DexedMLP1")


diff=net(x_test.cuda()).cpu()-y_test
diff=torch.pow(diff,2)
dim=list(diff.shape)
print(torch.sqrt(diff.sum(1).sum(0)/(dim[0]*dim[1])))
results1 = net(x_test.cuda()).cpu()
exportResultsPytorch(testIndex, y, results0, 'drive/My Drive/Comp562/DexedResults1.data')

Net(
  (hidden1): Linear(in_features=7205, out_features=20, bias=True)
  (hidden2): Linear(in_features=20, out_features=20, bias=True)
  (hidden3): Linear(in_features=20, out_features=20, bias=True)
  (hidden4): Linear(in_features=20, out_features=20, bias=True)
  (hidden5): Linear(in_features=20, out_features=20, bias=True)
  (hidden6): Linear(in_features=20, out_features=20, bias=True)
  (hidden7): Linear(in_features=20, out_features=20, bias=True)
  (hidden8): Linear(in_features=20, out_features=20, bias=True)
  (hidden9): Linear(in_features=20, out_features=20, bias=True)
  (hidden10): Linear(in_features=20, out_features=20, bias=True)
  (hidden11): Linear(in_features=20, out_features=20, bias=True)
  (hidden12): Linear(in_features=20, out_features=20, bias=True)
  (predict): Linear(in_features=20, out_features=155, bias=True)
)
Loss=0.6060
Loss=0.6016
Loss=0.5974
Loss=0.5932
Loss=0.5891
Loss=0.5850
Loss=0.5810
Loss=0.5769
Loss=0.5729
Loss=0.5688
Loss=0.5649
Loss=0.5609
Loss=0.5569